In [2]:
import boto3
import pandas as pd
import numpy as np
import psycopg2
import configparser

## Leemos archivo de configuración


In [16]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

## Nos identificamos con AWS


In [13]:
aws_rds_connection = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                              aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                              region_name='us-east-2')

## Verificamos instancias de AWS disponibles en el usuario


In [14]:
rds_instances_ids = []

aws_response = aws_rds_connection.describe_db_instances()

for response in aws_response['DBInstances']:
    rds_instances_ids.apped(response['DBInstanceIdentifier'])

print(f"Instancias Disponibles: {rds_instances_ids}")

Instancias Disponibles: []


## Creamos instancia de base de datos en AWS - PostGres


In [19]:
try:
    response = aws_rds_connection.create_db_instance(
        DBInstanceIdentifier=config.get('TRANSACC', 'DB_INSTANCE_ID'),
        DBName=config.get('TRANSACC', 'DB_NAME'),
        DBInstanceClass='db.t3.micro',
        Engine='postgres',
        MasterUsername=config.get('TRANSACC', 'DB_USER'),
        MasterUserPassword=config.get('TRANSACC', 'DB_PASSWORD'),
        Port=int(config.get('TRANSACC', 'DB_PORT')),
        PubliclyAccessible=True,
        VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
        AllocatedStorage=15
    )
    print(response)
except aws_rds_connection.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La instancia ya existe")
except Exception as ex:
    print("Error!", ex)

## Obtenemos el hostname de la instancia


In [21]:
try:
    instance = aws_rds_connection.describe_db_instances(DBInstanceIdentifier=config.get('TRANSACC', 'DB_INSTANCE_ID'))
    RDS_HOSTNAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOSTNAME)
except Exception as ex:
    print("Error!!", ex)

tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com


## Nos conectamos a la base de datos desde Python


In [23]:
import sql_queries
sql_queries.DDL_QUERY

'\nCREATE TABLE IF NOT EXISTS tipo_transacciones(\n    id_tipo_transac INT PRIMARY KEY,\n    tipo_transaccion VARCHAR(15) UNIQUE\n);\n\nCREATE TABLE IF NOT EXISTS divisas(\n    id_divisa INT PRIMARY KEY,\n    nombre_divisa VARCHAR(15) UNIQUE,\n    simbolo VARCHAR(2) UNIQUE\n);\n\nCREATE TABLE IF NOT EXISTS tipo_documentos(\n    id_tipo_documento INT PRIMARY KEY,\n    nombre_tipo_documento VARCHAR(15) UNIQUE\n);\n\nCREATE TABLE IF NOT EXISTS documentos(\n    id_documento INT PRIMARY KEY,\n    nombre_documento VARCHAR(50),\n    uri_documento VARCHAR(100),\n    fecha_hora_emision TIMESTAMP,\n    tipo_documento INT,\n\n    CONSTRAINT fk_tipo_documento\n        FOREIGN KEY (tipo_documento)\n            REFERENCES tipo_documentos(id_tipo_documento)\n\n);\n\nCREATE TABLE IF NOT EXISTS generos(\n    id_genero INT PRIMARY KEY,\n    genero VARCHAR(10) UNIQUE\n);\n\nCREATE TABLE IF NOT EXISTS sectores(\n    id_sector INT PRIMARY KEY,\n    nombre VARCHAR(50) UNIQUE,\n    latitud DOUBLE PRECISION,\

In [ ]:
try:
    db_pg_conn = psycopg2.connect(
        database=config.get('TRANSACC', 'DB_NAME'),
        user=config.get('TRANSACC', 'DB_USER'),
        password=config.get('TRANSACC', 'DB_PASSWORD'),
        host=RDS_HOSTNAME,
        port=int(config.get('TRANSACC', 'DB_PORT'))
    )
    cursor = db_pg_conn.cursor()
    cursor.execute(sql_queries.DDL_QUERY)
    db_pg_conn.commit()
    print("Base de Datos Creada Exitosamente")
except Exceptions as ex:
    print("Error!", ex)